<a href="https://colab.research.google.com/github/fender8185/Stock-predict/blob/main/lstmstock2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install FinMind==1.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 15.1 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24861 sha256=8790bf9b58f81da2458f64305ec9c6d0d0c83290f17d829a63364ada46c68522
  Stored in directory: /root/.cache/pip/wheels/d4/bd/c4/0dda911b5461ee856352cd8c9e16472229bb51c95e74b74c08
Successfully built ta


In [ ]:
import tensorflow as tf
import numpy as np
import FinMind
from FinMind.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from datetime import date


/usr/local/lib/python3.10/dist-packages/FinMind/strategies/base.py:423: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def final_stats(self) -> pd.Series():
/usr/local/lib/python3.10/dist-packages/FinMind/strategies/base.py:450: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def compare_market_stats(self) -> pd.Series():


In [ ]:
params = {
        'stock_id': '0050',
        'start_date': '2018-01-01',
        'end_date': date.today()

    }

data_loader = DataLoader()
df_holding = data_loader.taiwan_stock_holding_shares_per(**params)
df_holding = df_holding.pivot(index='date', columns='HoldingSharesLevel', values='percent').reset_index()
stock_data = data_loader.taiwan_stock_daily(**params)
df = df_holding.merge(stock_data, on='date', how='outer')
selected_columns = ['date', '1,000-5,000', '1-999', '10,001-15,000', '100,001-200,000',
       '15,001-20,000', '20,001-30,000', '200,001-400,000', '30,001-40,000',
       '40,001-50,000', '400,001-600,000', '5,001-10,000', '50,001-100,000',
       '600,001-800,000', '800,001-1,000,000', 'more than 1,000,001', 'close']

# Remove rows where all shareholder ratios are NaN or 0
df = df[selected_columns]

df[[ 'close']] = df[[ 'close']].replace(0, np.nan).fillna(method='ffill')

valid_ratio_indices = df[df.columns.difference(['date', 'close'])].dropna(how='all').index
df = df.loc[valid_ratio_indices]
all_dates = df['date'].values[1:]
df.drop(columns=['date'], inplace=True)
print(df)
numpy_array = df.values


     1,000-5,000  1-999  10,001-15,000  100,001-200,000  15,001-20,000  \
0          10.61   0.51           2.18             2.12           1.98   
1           9.88   0.53           1.95             2.16           1.79   
2           9.37   0.54           1.82             2.09           1.71   
3           9.65   0.56           1.82             2.24           1.75   
4          10.21   0.56           1.93             2.09           1.82   
..           ...    ...            ...              ...            ...   
287        19.75   4.11           4.34             1.80           2.90   
288        20.09   4.17           4.43             1.83           2.95   
289        20.57   4.26           4.55             1.87           3.05   
290        20.90   4.28           4.72             2.00           3.16   
291        20.82   4.26           4.72             2.00           3.15   

     20,001-30,000  200,001-400,000  30,001-40,000  40,001-50,000  \
0             2.34             2.68       

In [ ]:
price_diff = numpy_array[1:, -1] - numpy_array[:-1, -1]
y = price_diff
X = numpy_array[:-1]

split_index = int(len(X) * 0.8)
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]
train_dates = all_dates[:split_index]
test_dates = all_dates[split_index:]

# 數據歸一化
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).ravel()

# 重塑數據以供LSTM使用
timesteps = 10
X_train_reshaped = []
y_train_reshaped = []
for i in range(len(X_train_scaled) - timesteps):
    X_train_reshaped.append(X_train_scaled[i:i+timesteps])
    y_train_reshaped.append(y_train_scaled[i+timesteps])
X_train_reshaped = np.array(X_train_reshaped)
y_train_reshaped = np.array(y_train_reshaped)

X_test_reshaped = []
y_test_reshaped = []
for i in range(len(X_test_scaled) - timesteps):
    X_test_reshaped.append(X_test_scaled[i:i+timesteps])
    y_test_reshaped.append(y_test_scaled[i+timesteps])
X_test_reshaped = np.array(X_test_reshaped)
y_test_reshaped = np.array(y_test_reshaped)

# LSTM模型
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.fit(X_train_reshaped, y_train_reshaped, epochs=5000, batch_size=32)

# 預測
y_pred_train_scaled = model.predict(X_train_reshaped)
y_pred_test_scaled = model.predict(X_test_reshaped)

# 反縮放預測值回原始尺度
y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled)
y_pred_test = scaler_y.inverse_transform(y_pred_test_scaled)


Streaming output truncated to the last 5000 lines.
Epoch 2502/5000
7/7 [==============================] - 0s 38ms/step - loss: 9.0041e-06
Epoch 2503/5000
7/7 [==============================] - 0s 43ms/step - loss: 7.5635e-06
Epoch 2504/5000
7/7 [==============================] - 0s 31ms/step - loss: 7.7752e-06
Epoch 2505/5000
7/7 [==============================] - 0s 32ms/step - loss: 7.4108e-06
Epoch 2506/5000
7/7 [==============================] - 0s 31ms/step - loss: 5.0739e-06
Epoch 2507/5000
7/7 [==============================] - 0s 29ms/step - loss: 5.1631e-06
Epoch 2508/5000
7/7 [==============================] - 0s 28ms/step - loss: 7.8020e-06
Epoch 2509/5000
7/7 [==============================] - 0s 28ms/step - loss: 7.4764e-06
Epoch 2510/5000
7/7 [==============================] - 0s 38ms/step - loss: 6.4617e-06
Epoch 2511/5000
7/7 [==============================] - 0s 34ms/step - loss: 7.2533e-06
Epoch 2512/5000
7/7 [==============================] - 0s 28ms/step - loss: 8.1

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 調整日期以匹配預測值
adjusted_train_dates = train_dates[timesteps:]
adjusted_test_dates = test_dates[timesteps:]

fig = make_subplots(rows=1, cols=2, subplot_titles=('Training Data', 'Testing Data with Predictions'))

# 添加訓練數據到第一個子圖
fig.add_trace(go.Scatter(x=adjusted_train_dates, y=y_train[timesteps:], mode='lines', name='Training Data'), row=1, col=1)
fig.add_trace(go.Scatter(x=adjusted_train_dates, y=y_pred_train.flatten(), mode='lines', name='Predicted Training Data', marker=dict(symbol='circle-open')), row=1, col=1)

# 添加測試數據的真實值到第二個子圖
fig.add_trace(go.Scatter(x=adjusted_test_dates, y=y_test[timesteps:], mode='lines', name='Real Test Values'), row=1, col=2)
fig.add_trace(go.Scatter(x=adjusted_test_dates, y=y_pred_test.flatten(), mode='lines', name='Predicted Test Values', marker=dict(symbol='circle-open')), row=1, col=2)

fig.update_layout(hovermode='x unified')

# 顯示圖形
fig.show()
